In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ["THEANO_FLAGS"]="mode=FAST_RUN,device=cuda1,floatX=float32"

In [2]:
import keras

Using TensorFlow backend.


In [3]:
'''
#Train a simple deep CNN on the CIFAR10 small images dataset.

It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
import os

batch_size = 1000
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)


x_train shape: (50000, 3, 32, 32)
50000 train samples
10000 test samples


In [4]:
print(x_train.shape[1:])
print(type(x_train))

(3, 32, 32)
<type 'numpy.ndarray'>


In [5]:
import numpy as np

In [6]:
def rgb2gray(rgb):
    r, g, b = rgb[:,0,:,:], rgb[:,1,:,:], rgb[:,2,:,:]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    gray = np.expand_dims(gray, axis=1)
    print(gray.shape)
    return gray

In [7]:
x_train=rgb2gray(x_train)
x_test=rgb2gray(x_test)

(50000, 1, 32, 32)
(10000, 1, 32, 32)


In [8]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimage
plt.imshow(x_train[1,0,:,:], cmap='gray')

In [9]:
print(x_train[0].shape)

(1, 32, 32)


In [10]:
print(x_test.shape)

(10000, 1, 32, 32)


In [11]:
model = Sequential()
model.add(Convolution2D(6, 5, 5, border_mode='valid', input_shape=(1, 32, 32)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))   
model.add(Convolution2D(16, 5, 5, border_mode='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(120))
model.add(Activation('relu'))
model.add(Dense(84))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

/usr/local/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), padding="valid", input_shape=(1, 32, 32...)`
  
/usr/local/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (5, 5), padding="valid")`
  """


In [12]:
print(x_train[0].shape)
print(x_train[0,:,:,:])

(1, 32, 32)
[[[ 61.2111  44.9847  48.023  ... 137.0222 130.4358 128.7672]
  [ 18.8024   0.      10.0762 ...  94.6907  89.9901  94.0328]
  [ 23.9545   8.8914  31.4071 ...  90.2782  90.876   80.2191]
  ...
  [172.9052 153.7659 156.6532 ... 133.875   35.7334  38.0797]
  [146.339  128.6987 143.5434 ... 152.5896  69.2633  59.7957]
  [150.6573 136.6542 146.7751 ... 188.5304 123.9759  98.9767]]]


In [13]:
y_test.shape[1]

10

In [14]:
# Loss function
def neg_log_likelihood_loss(y_true, y_pred):
    ##sep = y_pred.shape[1] // 2
    mu, logvar = y_pred[:, :10], y_pred[:, 10:]
    return K.sum(0.5*(logvar+np.log(2*np.pi)+K.square((y_true-mu)/K.exp(0.5*logvar))), axis=-1)

In [15]:
# initiate RMSprop optimizer
sgd = SGD(lr=0.01, decay=0, momentum=0.5, nesterov=True)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['categorical_crossentropy', 'accuracy']
             )

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [17]:
type(x_train)

numpy.ndarray

In [27]:
model.fit(x_train, y_train,
         batch_size=batch_size,
         nb_epoch=10,
         validation_data=(x_test, y_test),
         shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 0s - loss: 2.3025 - categorical_crossentropy: 2.3025 - acc: 0.1114 - val_loss: 2.3025 - val_categorical_crossentropy: 2.3025 - val_acc: 0.1065
Epoch 2/10
50000/50000 [==============================] - 0s - loss: 2.3025 - categorical_crossentropy: 2.3025 - acc: 0.1056 - val_loss: 2.3025 - val_categorical_crossentropy: 2.3025 - val_acc: 0.1000
Epoch 3/10
50000/50000 [==============================] - 0s - loss: 2.3025 - categorical_crossentropy: 2.3025 - acc: 0.1109 - val_loss: 2.3025 - val_categorical_crossentropy: 2.3025 - val_acc: 0.1001
Epoch 4/10
50000/50000 [==============================] - 0s - loss: 2.3025 - categorical_crossentropy: 2.3025 - acc: 0.1018 - val_loss: 2.3025 - val_categorical_crossentropy: 2.3025 - val_acc: 0.1000
Epoch 5/10
50000/50000 [==============================] - 0s - loss: 2.3025 - categorical_crossentropy: 2.3025 - acc: 0.1101 - val_loss: 2.3025 - 

In [16]:
import time
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              nb_epoch=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
    start_time = time.time()
    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                        batch_size=batch_size),
                        #steps_per_epoch=x_train.shape[0] // batch_size,
                        samples_per_epoch=x_train.shape[0],
                        nb_epoch=epochs,
                        validation_data=(x_test, y_test))
print("--- Training Time - %s seconds ---" % (time.time() - start_time))

Using real-time data augmentation.


/usr/local/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:34: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=100, validation_data=(array([[[..., steps_per_epoch=50)`


Epoch 1/100
50/50 [==============================] - 6s - loss: 2.3017 - categorical_crossentropy: 2.3017 - acc: 0.1039 - val_loss: 2.2990 - val_categorical_crossentropy: 2.2990 - val_acc: 0.1079
Epoch 2/100
50/50 [==============================] - 6s - loss: 2.2982 - categorical_crossentropy: 2.2982 - acc: 0.1177 - val_loss: 2.2969 - val_categorical_crossentropy: 2.2969 - val_acc: 0.1264
Epoch 3/100
50/50 [==============================] - 6s - loss: 2.2959 - categorical_crossentropy: 2.2959 - acc: 0.1329 - val_loss: 2.2945 - val_categorical_crossentropy: 2.2945 - val_acc: 0.1367
Epoch 4/100
50/50 [==============================] - 6s - loss: 2.2934 - categorical_crossentropy: 2.2934 - acc: 0.1393 - val_loss: 2.2914 - val_categorical_crossentropy: 2.2914 - val_acc: 0.1445
Epoch 5/100
50/50 [==============================] - 6s - loss: 2.2900 - categorical_crossentropy: 2.2900 - acc: 0.1453 - val_loss: 2.2871 - val_categorical_crossentropy: 2.2871 - val_acc: 0.1481
Epoch 6/100
50/50 [=

50/50 [==============================] - 6s - loss: 1.7708 - categorical_crossentropy: 1.7708 - acc: 0.3678 - val_loss: 1.7174 - val_categorical_crossentropy: 1.7174 - val_acc: 0.3878
Epoch 43/100
50/50 [==============================] - 6s - loss: 1.7597 - categorical_crossentropy: 1.7597 - acc: 0.3721 - val_loss: 1.6825 - val_categorical_crossentropy: 1.6825 - val_acc: 0.4000
Epoch 44/100
50/50 [==============================] - 6s - loss: 1.7568 - categorical_crossentropy: 1.7568 - acc: 0.3691 - val_loss: 1.6663 - val_categorical_crossentropy: 1.6663 - val_acc: 0.4038
Epoch 45/100
50/50 [==============================] - 6s - loss: 1.7543 - categorical_crossentropy: 1.7543 - acc: 0.3719 - val_loss: 1.6686 - val_categorical_crossentropy: 1.6686 - val_acc: 0.4023
Epoch 46/100
50/50 [==============================] - 6s - loss: 1.7411 - categorical_crossentropy: 1.7411 - acc: 0.3764 - val_loss: 1.6681 - val_categorical_crossentropy: 1.6681 - val_acc: 0.4055
Epoch 47/100
50/50 [========

50/50 [==============================] - 6s - loss: 1.5923 - categorical_crossentropy: 1.5923 - acc: 0.4336 - val_loss: 1.5065 - val_categorical_crossentropy: 1.5065 - val_acc: 0.4610
Epoch 84/100
50/50 [==============================] - 6s - loss: 1.5848 - categorical_crossentropy: 1.5848 - acc: 0.4350 - val_loss: 1.5287 - val_categorical_crossentropy: 1.5287 - val_acc: 0.4516
Epoch 85/100
50/50 [==============================] - 6s - loss: 1.5930 - categorical_crossentropy: 1.5930 - acc: 0.4337 - val_loss: 1.4972 - val_categorical_crossentropy: 1.4972 - val_acc: 0.4617
Epoch 86/100
50/50 [==============================] - 6s - loss: 1.5797 - categorical_crossentropy: 1.5797 - acc: 0.4396 - val_loss: 1.4893 - val_categorical_crossentropy: 1.4893 - val_acc: 0.4672
Epoch 87/100
50/50 [==============================] - 6s - loss: 1.5735 - categorical_crossentropy: 1.5735 - acc: 0.4405 - val_loss: 1.5139 - val_categorical_crossentropy: 1.5139 - val_acc: 0.4603
Epoch 88/100
50/50 [========

In [36]:
# Evaluate model with test data set and share sample prediction results
# evaluation = model.evaluate_generator(datagen.flow(x_test, y_test,
#                                       batch_size=batch_size),
#                                       steps=x_test.shape[0] // batch_size)
print(x_test.shape, y_test.shape)
evaluation = model.evaluate_generator(datagen.flow(x_test, y_test,batch_size=batch_size),
                                      val_samples=x_test.shape[0],
                                      show_accuracy = True,
                                        verbose=1)
print(evaluation)
print('Model Accuracy = %.2f, Model loss = %.2f ' % (evaluation[2], evaluation[1]))

(10000, 1, 32, 32) (10000, 10)
1000/1000 [==============================] - 0s
[2.3025865077972414, 0.1]


IndexError: list index out of range

In [175]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
##model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
#scores = model.evaluate(x_test, y_test,show_accuracy=True, verbose=1, batch_size=1000)
scores = model.evaluate(x_test, y_test, verbose=1, batch_size=2000)
print(scores)
#print('Test loss:', scores[0])
#print('Test accuracy:', scores[1])

Saved trained model at /root/shared/saved_models/keras_cifar10_trained_model.h5 
 2000/10000 [=====>........................] - ETA: 0s[1.5061332702636718, 1.5061332702636718, 0.47669999599456786]


In [77]:
print(batch_size)

64
